In [ ]:
!pip install confluent_kafka

In [ ]:
import json
import sys
import time
import socket
from confluent_kafka import Consumer, KafkaError

def msg_process(msg):
    # Print the current time and the message.
    time_start = time.strftime("%Y-%m-%d %H:%M:%S")
    val = msg.value()
    dval = json.loads(val)
    print(time_start, dval)

# Kafka consumer configuration
consumer_conf = {
    'bootstrap.servers': 'PLAINTEXT://kafka:9092',  # Replace with your Kafka broker address
    'group.id': socket.gethostname(),
    'auto.offset.reset': 'earliest'
}

# Create Kafka consumer
consumer = Consumer(consumer_conf)

# Subscribe to the topic
topic = "other_topic"  # Replace with your Kafka topic
consumer.subscribe([topic])

# Consume messages
try:
    while True:
        msg = consumer.poll(1.0)
        if msg is None:
            continue

        if msg.error():
            if msg.error().code() == KafkaError._PARTITION_EOF:
                # End of partition event
                sys.stderr.write('%% %s [%d] reached end at offset %d\n' %
                                 (msg.topic(), msg.partition(), msg.offset()))
            elif msg.error().code() == KafkaError.UNKNOWN_TOPIC_OR_PART:
                sys.stderr.write('Topic unknown, creating %s topic\n' %
                                 (topic))
            elif msg.error():
                raise KafkaException(msg.error())
        else:
            msg_process(msg)

except KeyboardInterrupt:
    pass

finally:
    # Close down consumer to commit final offsets.
    consumer.close()
